In [1]:
import pandas as pd

from configparser import ConfigParser
from mysql.connector import MySQLConnection,Error

In [2]:
items = pd.read_parquet('../../../../../Datasets/Processed/books_SE_v5.parquet')

In [11]:
len(items)

2113033

In [3]:
dataframe_columns = [
    "book_id",
    "title_without_series",
    "mod_title",
    "title",
    "mod_title_without_series",
    "description",
    "ratings_count",
    "average_rating",
    "num_pages",
    "publication_day",
    "publication_month",
    "publication_year",
    "isbn",
    "isbn13",
    "publisher",
    "country_code",
    "language_code",
    "url",
    "image_url",
    "link"
]

**Function to fetch one row at a time from the dataframe**

In [4]:
def getrow(df,dataframe_columns=dataframe_columns,row_num=0):
    return df.loc[row_num,dataframe_columns].values

**Test the function**

In [21]:
getrow(df=items,row_num=0)

array(['5333265', 'W.C. Fields: A Life on Film',
       'wc fields a life on film', 'W.C. Fields: A Life on Film',
       'wc fields a life on film', '', 3, 4.0, 256, 1, 9, 1984,
       '0312853122', '9780312853129', "St. Martin's Press", 'US', '',
       'https://www.goodreads.com/book/show/5333265-w-c-fields',
       'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
       'https://www.goodreads.com/book/show/5333265-w-c-fields'],
      dtype=object)

**Function to change the datatype**
- We will convert `numpy.int64` and `numpy.float64` into `int` and `float`

In [5]:
def tointfloat(data):
    data[8] = int(data[8])
    data[9] = float(data[9])
    data[10] = int(data[10])
    data[11] = int(data[11])
    data[12] = int(data[12])
    data[13] = int(data[13])

    return data

**function for `sp_create_book` proc**

In [6]:
def sp_create_book(data,cs,proc_name="sp_create_book",initial=[0,1],end=[0]):
    
    procs_args_complete = []

    procs_args_complete.extend(initial)
    procs_args_complete.extend(data)
    procs_args_complete.extend(end)

    upload_details = cs.callproc(procname=proc_name, args=tointfloat(procs_args_complete))

    return upload_details

**Config reading and parsing**

In [7]:
# reading database credentials from config.ini file

def read(filename='config.ini',section='mysql'):
    parser=ConfigParser()
    parser.read(filename)
    
    db={}
    
    if parser.has_section(section):
        items=parser.items(section)
        for item in items:
            db[item[0]]=item[1]
    else:
        raise Exception(f'{section} not found in file {filename}')
    return db 


print(read(filename="config.ini",section="mysql"))

{'host': '127.0.0.1', 'port': '99966', 'database': 'books', 'user': 'root', 'password': 'Milla123!'}


**Connecting with MySQL/MariaDB database server and getting the connection and cursor object**

In [8]:
def connect(creds):
    con=None
    try:
        print('Connecting to MySQL database...')
        con=MySQLConnection(**creds)
        
        
        if con.is_connected():
            print('Connection established')
            cus = con.cursor(buffered=True)
        else:
            print('Connection failed')
            
    except Error as e:
        print(e)
    finally:
        return con,cus
    
cn,cs=connect(creds=read(filename="config.ini",section="mysql"))

Connecting to MySQL database...
Connection established


### Uploading the Complete Data into the Database

In [10]:
for i in range(items.shape[0]):
    record = getrow(df=items,dataframe_columns=dataframe_columns,row_num=i)
    sp_create_book(data=record,cs=cs,proc_name="sp_create_book",initial=[0,1],end=[0])

    if ((i%100000) == 0):
        cn.commit()

cn.commit()